# HomeWork 7
На цьому тижні ми вивчили як працюють рекомендаційні системи. Пропонуємо вам познайомитись з бібліотекою surprise, котра якраз є по суті додатком до знайомої нам бібліотеки scikit-learn для тренування моделей рекомендаційних систем.

Візьміть датасет movielens і побудуйте модель матричної факторизації. У даній бібліотеці він має назву SVD. Підберіть найкращі параметри за допомогою крос-валідації, також поекспериментуйте з іншими алгоритмами розрахунків (SVD++, NMF) і оберіть той, який буде оптимальним.

Підказки як саме побудувати дану модель ви знайдете в документації до даної бібліотеки.

In [2]:
from itertools import combinations
import math as m
from pathlib import Path
import pickle
import random
from typing import Union, Optional

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
# from kneed import KneeLocator
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.svm import SVC
# import tsfresh  # poetry add tsfresh
%matplotlib inline

__OBTAIN & SCRUB__

- links.csv - movieId, imdbId, tmdbId
- movies.csv - movieId, title, genres
- ratings.csv - userId, movieId, rating, timestamp
- tags.csv - userId, movieId, tag, timestamp

In [3]:
def read_from_csvfile(file: Path) -> pd.DataFrame:
    """Read content from csv-file and return dataframe from content."""
    df = pd.read_csv(file)

    return df

In [8]:
links = read_from_csvfile('/content/ml/links.csv')
movies = read_from_csvfile('/content/ml/movies.csv')
ratings = read_from_csvfile('/content/ml/ratings.csv')
tags = read_from_csvfile('/content/ml/tags.csv')

In [9]:
links.tail(3)

,movieId,imdbId,tmdbId
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0
9741,193609,101726,37891.0


In [10]:
movies.tail(3)

,movieId,title,genres
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation
9741,193609,Andrew Dice Clay: Dice Rules (1991),Comedy


In [34]:
ratings.tail(3)

,userId,movieId,rating,timestamp
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352
100835,610,170875,3.0,1493846415


In [12]:
tags.tail(3)

,userId,movieId,tag,timestamp
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978
3682,610,168248,Heroic Bloodshed,1493844270


- Використовуємо лише датасет рейтингу, для найпростішої системи?

__MODEL__

- модель матричної факторизації. У даній бібліотеці він має назву SVD

- https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD
- https://www.piwheels.org/project/scikit-surprise/
- https://pypi.org/project/scikit-surprise/
- https://github.com/NicolasHug/Surprise

!pip install matplotlib

In [14]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=2811647 sha256=a0c11ba082ec62634e44c6efcf6af2609e277b068a1d142f09058ef54df86ce0
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [15]:
from surprise.prediction_algorithms.matrix_factorization import SVD  # poetry add surprise
# poetry add scikit-surprise==1.1.3 --use-pep517
# !!!! Installing scikit-surprise (1.1.3): Failed
# colab - Ok

- https://surprise.readthedocs.io/en/stable/getting_started.html#cross-validate-example

In [33]:
# from surprise import SVD
# from surprise import Dataset
# from surprise.model_selection import cross_validate

# # Load the movielens-100k dataset (download it if needed).
# data = Dataset.load_builtin('ml-100k')

# # Use the famous SVD algorithm.
# algo = SVD()

# # Run 5-fold cross-validation and print results.
# cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

- https://surprise.readthedocs.io/en/stable/model_selection.html

In [28]:
# cross_validate(
#                algo,
#                data,
#                measures=['rmse', 'mae'],
#                cv=None,
#                return_train_measures=False,
#                n_jobs=1,
#                pre_dispatch='2*n_jobs',
#                verbose=False
#                )

- https://surprise.readthedocs.io/en/stable/getting_started.html?highlight=Dataset#use-a-custom-dataset

In [ ]:
from surprise import SVD
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate

Column names are irrelevant, but specify only one rating per line, and each line needs to respect the following structure:<br> `user ; item ; rating ; [timestamp]`

In [35]:
df = ratings[['userId', 'movieId', 'rating']]

In [41]:
df.tail(3)

,userId,movieId,rating
100833,610,168250,5.0
100834,610,168252,5.0
100835,610,170875,3.0


In [48]:
df['rating'].max(), df['rating'].min(), df['rating'].mean(), df['rating'].std()

(5.0, 0.5, 3.501556983616962, 1.042529239060635)

0.5 мінімальний рейтинг, тому логічно що межа мінімума 0:

- https://surprise.readthedocs.io/en/stable/reader.html?highlight=Reader#surprise.reader.Reader

In [42]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(0, 5))

In [43]:
# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

In [45]:
algo2 = SVD(
            n_factors=1,
            n_epochs=10,
            biased=True,
            init_mean=2.5,
            init_std_dev=0.5,
            lr_all=0.005,
            reg_all=0.02,
            lr_bu=None,
            lr_bi=None,
            lr_pu=None,
            lr_qi=None,
            reg_bu=None,
            reg_bi=None,
            reg_pu=None,
            reg_qi=None,
            random_state=None,
            verbose=False
            )

In [46]:
# Run 6-fold cross-validation and print results.
cross_validate(algo2, data, measures=['RMSE', 'MAE'], cv=6, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 6 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Mean    Std     
RMSE (testset)    0.8773  0.8767  0.8783  0.8754  0.8740  0.8764  0.8764  0.0014  
MAE (testset)     0.6781  0.6737  0.6791  0.6732  0.6726  0.6762  0.6755  0.0025  
Fit time          0.65    0.52    0.55    0.50    0.54    0.55    0.55    0.05    
Test time         0.23    0.14    0.12    0.18    0.14    0.33    0.19    0.07    


{'test_rmse': array([0.87733225, 0.87673298, 0.8783383 , 0.87544726, 0.87404887,
        0.87642241]),
 'test_mae': array([0.67806792, 0.67367781, 0.67905478, 0.67322409, 0.67264009,
        0.67622364]),
 'fit_time': (0.6548593044281006,
  0.5241212844848633,
  0.5488119125366211,
  0.5006754398345947,
  0.5435709953308105,
  0.5492165088653564),
 'test_time': (0.2271735668182373,
  0.13544225692749023,
  0.12300491333007812,
  0.18457531929016113,
  0.13525629043579102,
  0.33479881286621094)}

- Підберіть найкращі параметри за допомогою крос-валідації


- поекспериментуйте з іншими алгоритмами розрахунків (SVD++, NMF) і оберіть той, який буде оптимальним

In [49]:
algo_SVD = [
            SVD(
                n_factors=nfs,
                n_epochs=eph,
                init_mean=df['rating'].mean(), # (0+5)/2 ...
                init_std_dev=df['rating'].std()
                )
            for nfs in (1, 5, 10, 20, 40)
            for eph in (5, 10, 20)
            ]

In [53]:
assessment_SVD = [cross_validate(alg, data, measures=['RMSE', 'MAE'], cv=cv, verbose=True) for alg in algo_SVD for cv in (3, 5, 6)]

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9522  0.9521  0.9520  0.9521  0.0001  
MAE (testset)     0.7412  0.7400  0.7401  0.7404  0.0006  
Fit time          0.18    0.16    0.24    0.19    0.03    
Test time         0.24    0.50    0.20    0.31    0.13    
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9391  0.9395  0.9289  0.9361  0.9384  0.9364  0.0039  
MAE (testset)     0.7290  0.7261  0.7181  0.7270  0.7271  0.7255  0.0038  
Fit time          0.20    0.24    0.21    0.22    0.20    0.21    0.01    
Test time         0.13    0.16    0.24    0.49    0.13    0.23    0.13    
Evaluating RMSE, MAE of algorithm SVD on 6 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Mean    Std     
RMSE (testset)    0.9296  0.9428  0.9288  0.9295  0.9309  0.9309  0.9321  0.0048  
MA

In [54]:
assessment_SVD

[{'test_rmse': array([0.9521897 , 0.95213411, 0.95198202]),
  'test_mae': array([0.7411969 , 0.73996825, 0.74010393]),
  'fit_time': (0.17867398262023926, 0.16194820404052734, 0.23865342140197754),
  'test_time': (0.23746562004089355, 0.500664472579956, 0.2035691738128662)},
 {'test_rmse': array([0.9390907 , 0.93951863, 0.92888794, 0.93609506, 0.93841549]),
  'test_mae': array([0.72900144, 0.72608083, 0.71807638, 0.72699751, 0.72710678]),
  'fit_time': (0.19878482818603516,
   0.23848176002502441,
   0.20652985572814941,
   0.21763134002685547,
   0.20424652099609375),
  'test_time': (0.1345353126525879,
   0.15815067291259766,
   0.2418684959411621,
   0.4863271713256836,
   0.1288597583770752)},
 {'test_rmse': array([0.92962182, 0.94277939, 0.92877477, 0.92952813, 0.93093632,
         0.9308927 ]),
  'test_mae': array([0.7234972 , 0.73165471, 0.72013719, 0.72457728, 0.7247373 ,
         0.71907338]),
  'fit_time': (0.23456525802612305,
   0.2182455062866211,
   0.18857455253601074,
 

...

In [39]:
# def cofiCostFunc(params, Y, R, num_users, num_movies, num_features, lambda_=0.0):
#     """
#     Collaborative filtering cost function.
#     Parameters
#       params : array_like
#         The parameters which will be optimized. This is a one dimensional vector of shape (num_movies x num_users, 1). It is the concatenation of the feature vectors X and parameters Theta.
#       Y : array_like
#         A matrix of shape (num_movies x num_users) of user ratings of movies.
#       R : array_like
#         A (num_movies x num_users) matrix, where R[i, j] = 1 if the i-th movie was rated by the j-th user.
#       num_users : int
#         Total number of users.
#       num_movies : int
#         Total number of movies.
#       num_features : int
#         Number of features to learn.
#       lambda_ : float, optional
#         The regularization coefficient.

#     Returns
#       J : float
#        The value of the cost function at the given params.
#       grad : array_like
#         The gradient vector of the cost function at the given params. grad has a shape (num_movies x num_users, 1)

#     Instructions
#       Compute the cost function and gradient for collaborative filtering. Concretely, you should first implement the cost function (without regularization) and make sure it is matches our costs. After that, you should implement thegradient and use the checkCostFunction routine to check that the gradient is correct. Finally, you should implement regularization.

#     Notes
#       -	The input params will be unraveled into the two matrices:
#       X : (num_movies x num_features) matrix of movie features Theta : (num_users x num_features) matrix of user features
#       -	You should set the following variables correctly:
#       X_grad : (num_movies x num_features) matrix, containing the partial derivatives w.r.t. to each element of X Theta_grad : (num_users x num_features) matrix, containing the
#       partial derivatives w.r.t. to each element of Theta
#       -	The returned gradient will be the concatenation of the raveled gradients X_grad and Theta_grad.
#     """
#     # Unfold the U and W matrices from params
#     X = params[:num_movies*num_features].reshape(num_movies, num_features)
#     Theta = params[num_movies*num_features:].reshape(num_users, num_features)

#     #	You need to return the following values correctly
#     J = 0
#     X_grad = np.zeros(X.shape)
#     Theta_grad = np.zeros(Theta.shape)

#     #	====================== YOUR CODE HERE ======================

#     J = (1 / 2) * np.sum(np.square((X.dot(Theta.T) - Y) * R)) + (lambda_ / 2) * np.sum(np.square(X)) + (lambda_ / 2) * np.sum(np.square(Theta))
#     for i in range(R.shape[0]):
#         idx = np.where(R[i, :] == 1)[0]
#         Theta_temp = Theta[idx, :]
#         Y_temp = Y[i, idx]
#         X_grad[i, :] = np.dot(np.dot(X[i, :], Theta_temp.T) - Y_temp, Theta_temp) + lambda_ * X[i, :]

#     for j in range(R.shape[1]):
#         idx = np.where(R[:, j] == 1)[0]
#         X_temp = X[idx, :]
#         Y_temp = Y[idx, j]
#         Theta_grad[j, :] = np.dot(np.dot(X_temp, Theta[j, :]) - Y_temp, X_temp) + lambda_ * Theta[j, :]

#     #	=============================================================

#     grad = np.concatenate([X_grad.ravel(), Theta_grad.ravelQ])

#     return 3, grad
